In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

data = pd.read_csv('../Data/Final_table.csv')

#  Performing Descision Tree Linear Regression
### Splitting dataframe into train and test datasets

In [77]:
feature_var = data.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28]]
target_var = data['DPL_historical_da']

X_train, X_test, y_train, y_test = train_test_split(feature_var, target_var, test_size=0.2, random_state=156)

### Running regression tree model with depth of 10

In [78]:
model = tree.DecisionTreeRegressor(max_depth = 10, random_state = 156)
model = model.fit(X_train, y_train)

model_text = tree.export_text(model, feature_names=list(X_train.columns))
print(model_text)

|--- DOM_forecast <= 16945.49
|   |--- Henry Hub Natural Gas Spot Price (Dollars per Million Btu) <= 4.88
|   |   |--- APS_forecast <= 5871.60
|   |   |   |--- Henry Hub Natural Gas Spot Price (Dollars per Million Btu) <= 4.45
|   |   |   |   |--- APS_forecast <= 5174.21
|   |   |   |   |   |--- forecast_gen_outage_mw_other <= 19992.58
|   |   |   |   |   |   |--- APS_forecast <= 4500.52
|   |   |   |   |   |   |   |--- WEST_wind <= 6269.98
|   |   |   |   |   |   |   |   |--- APS_forecast <= 4249.85
|   |   |   |   |   |   |   |   |   |--- MIDATL_wind <= 446.14
|   |   |   |   |   |   |   |   |   |   |--- value: [12.83]
|   |   |   |   |   |   |   |   |   |--- MIDATL_wind >  446.14
|   |   |   |   |   |   |   |   |   |   |--- value: [15.78]
|   |   |   |   |   |   |   |   |--- APS_forecast >  4249.85
|   |   |   |   |   |   |   |   |   |--- DAY_forecast <= 1551.10
|   |   |   |   |   |   |   |   |   |   |--- value: [16.92]
|   |   |   |   |   |   |   |   |   |--- DAY_forecast >  1551.

### Determining importance of feature columns

In [79]:
fi = model.feature_importances_

names = X_train.columns
importance_dict = dict(zip(names, fi))
sorted_importance = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

print("Feature Importance:")
for feature, importance in sorted_importance:
    print(f"{feature}: {importance}")

Feature Importance:
DOM_forecast: 0.26468345497402707
Henry Hub Natural Gas Spot Price (Dollars per Million Btu): 0.24876494530378598
MIDATL_forecast: 0.07239306633077387
OTHER_wind: 0.05914316696728936
forecast_gen_outage_mw_west: 0.03565854576434285
MIDATL_wind: 0.03396500653870998
WEST_wind: 0.03276612685172521
APS_forecast: 0.029445702624922132
forecast_gen_outage_mw_other: 0.02761518930906463
EKPC_forecast: 0.02515401783484795
forecast_gen_outage_mw_rto: 0.025053645280615124
SOUTH_wind: 0.021913695999337554
RFC_solar: 0.017957300651340647
DAY_forecast: 0.017199381763092523
MIDATL_solar: 0.013854193012598088
SOUTH_solar: 0.012328277440462342
DEOK_forecast: 0.008929336389489742
RFC_wind: 0.008250287829421362
ATSI_forecast: 0.007993333057259727
RTO_wind: 0.006802600316499277
AEP_forecast: 0.005344106122936581
COMED_forecast: 0.005079589988988828
RTO_forecast: 0.004917958539810697
RTO_solar: 0.00436481443667758
DUQ_forecast: 0.004054356774409311
WEST_solar: 0.00381402137936651
OTHER_s

### Finding MSE and R^2

In [80]:
preds = model.predict(X_test)
print(mean_squared_error(y_test, preds), r2_score(y_test, preds))

1003.8902916878261 0.608319871361817


### Determining Optimal Depth between 1 and 30

In [81]:
mse = {'k':[], 'train_mse':[], 'test_mse':[]}
for k in range(1,30):
    print("Fit with max_depth:", k, end='\r', flush=True)
    
    model = tree.DecisionTreeRegressor(max_depth=k, random_state= 156)
    model = model.fit(X_train, y_train)
    preds_train = model.predict(X_train)
    preds_test = model.predict(X_test)

    mse['k'].append(k)
    mse['train_mse'].append(mean_squared_error(y_train, preds_train))
    mse['test_mse'].append(mean_squared_error(y_test, preds_test))
    
idx = mse['test_mse'].index(min(mse['test_mse']))
print('Depth of the model yielding minimum test MSE is:', mse['k'][idx])
print('Optimized model has MSE:', min(mse['test_mse']))

Depth of the model yielding minimum test MSE is: 15
Optimized model has MSE: 925.9028669409839


### Running regression tree model with optimal depth of 25

In [82]:
model = tree.DecisionTreeRegressor(max_depth = 15, random_state = 156)
model = model.fit(X_train, y_train)

model_text = tree.export_text(model, feature_names=list(X_train.columns))
print(model_text)

|--- DOM_forecast <= 16945.49
|   |--- Henry Hub Natural Gas Spot Price (Dollars per Million Btu) <= 4.88
|   |   |--- APS_forecast <= 5871.60
|   |   |   |--- Henry Hub Natural Gas Spot Price (Dollars per Million Btu) <= 4.45
|   |   |   |   |--- APS_forecast <= 5174.21
|   |   |   |   |   |--- forecast_gen_outage_mw_other <= 19992.58
|   |   |   |   |   |   |--- APS_forecast <= 4500.52
|   |   |   |   |   |   |   |--- WEST_wind <= 6269.98
|   |   |   |   |   |   |   |   |--- APS_forecast <= 4249.85
|   |   |   |   |   |   |   |   |   |--- MIDATL_wind <= 446.14
|   |   |   |   |   |   |   |   |   |   |--- forecast_gen_outage_mw_west <= 16829.97
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 5
|   |   |   |   |   |   |   |   |   |   |--- forecast_gen_outage_mw_west >  16829.97
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 5
|   |   |   |   |   |   |   |   |   |--- MIDATL_wind >  446.14
|   |   |   |   |   |   |   |   |   |   |---

### Determining importance of feature columns

In [83]:
fi = model.feature_importances_

names = X_train.columns
importance_dict = dict(zip(names, fi))
sorted_importance = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

print("Feature Importance:")
for feature, importance in sorted_importance:
    print(f"{feature}: {importance}")

Feature Importance:
DOM_forecast: 0.2453183247502944
Henry Hub Natural Gas Spot Price (Dollars per Million Btu): 0.2428100804876364
MIDATL_forecast: 0.072804466760047
OTHER_wind: 0.06128131561517162
MIDATL_wind: 0.03541469725165324
forecast_gen_outage_mw_west: 0.03172614353850579
EKPC_forecast: 0.031641125440062354
APS_forecast: 0.030820746429552
WEST_wind: 0.028644365953447406
forecast_gen_outage_mw_other: 0.028538518177562852
SOUTH_wind: 0.021718388216829483
forecast_gen_outage_mw_rto: 0.02094861868896996
DAY_forecast: 0.01761096818032303
RFC_solar: 0.01680872657575419
MIDATL_solar: 0.01584433054146511
SOUTH_solar: 0.013097504887769205
RFC_wind: 0.012363975738967714
DEOK_forecast: 0.010076920397822011
DUQ_forecast: 0.009635521697471222
COMED_forecast: 0.009579596607328439
ATSI_forecast: 0.008644517354004781
RTO_wind: 0.008456295420454686
AEP_forecast: 0.00805629173790001
RTO_forecast: 0.005453242088345882
RTO_solar: 0.004567823729322027
OTHER_solar: 0.004413670043499089
WEST_solar: 0

### Finding MSE and R^2

In [84]:
preds = model.predict(X_test)
print(mean_squared_error(y_test, preds), r2_score(y_test, preds))

925.9028669409839 0.6387476230891964


### Evaluating accuracy of the model based on comparing predicted price vs actual price of a specific date
The date chosen was March 2nd 11am (using forecast data from March 1st 11am)

In [86]:
columns = X_train.columns 

input_data = [1149.953, 226.95, 2116.221, 4487.92, 2371.699, 966.268, 449.758, 23.421, 2664.284, 2840.967,
              176.683, 2214.526, 15989, 6107, 8218, 10712, 2187, 3224, 14292, 1568, 1882, 29929, 93780, 0, 0, 0, 1.47]

# Correctly creating a DataFrame from the input_data
input_df = pd.DataFrame([input_data], columns=columns)

predicted_value = model.predict(input_df)
print("Predicted price:", predicted_value[0])

print("Actual price on March 2nd 11AM: 27.05")
print("Accuracy of Model: {}".format(predicted_value / 27.05))


Predicted price: 23.189285714285713
Actual price on March 2nd 11AM: 27.05
Accuracy of Model: [0.85727489]
